# 熟悉pytorch

## 学习如何使用pytorch将数据在GPU和CPU间切换
    - 模型的转换
    - model = Model()
    -model.to(device)
    -数据的转换
    -dada = data.to(device)

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 学习如何使用pytorch实现三层全连接模型
    - 三层全连接模型包括一个输入层、两个隐藏层和一个输出层
    - 模型定义使用torch.nn.Module

In [39]:
import torch
import torch.nn as nn

class ThreeLayerFC(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(ThreeLayerFC, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 学习如何使用pytorch搭建一个模型配置框架，包括：数据读取、转换、超参数定义。训练过程和测试显示等

In [40]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### 数据转换

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0), (0.5, 0))])
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False,)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

### 超参数定义

In [31]:
def main():
    learning_rate = 0.001
    batch_size = 64
    num_epochs = 10
    input_size = 28 * 28
    hidden_size = 128
    output_size = 10

### 训练过程和测试显示

In [9]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx , (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
             print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

In [55]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

## 综合代码

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class ThreeLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerFC, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

def main():
    learning_rate = 0.001
    batch_size = 64
    num_epochs = 10
    input_size = 784
    hidden_size = 128
    output_size = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model = ThreeLayerFC(input_size, hidden_size, output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(1, num_epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
if __name__ == '__main__':
    main()

## 评价

代码结构总体没什么问题，不过有几个小的比较奇怪的地方，不知道你有没有留意，我先标记出来以供参考和讨论：

### 超参数定义

超参数定义的单独的代码块中，用main函数封装了一堆超参数。不过如果脱离整体的代码来跑的话，封装在函数里的变量会成为局部变量，函数外部无法读取。

一般情况下有几种封装超参数比较好的方法可供参考：

- 做一个配置文件`config.py`，配置文件里写入各种参数，在需要调用的代码中使用`from config import *`即可将所有超参数引入此代码文件的全局参数空间
- 使用字典，示例如下

  ```python
  hp = {
      lr = 0.001,
      batch_size = 64,
  }
  ```

另外，模型结构也可以算作是超参数，里面的层数、每层的神经元个数等，都可以使用超参数进行调节。比如一个全连接层的超参数可以表示为如下示例

```python
hp = {
    neural_numlist = [784, 128, 128, 10]
}
```

### test

在最后的综合代码当中，test和train一起放入了batch的循环当中，这是否有必要呢？毕竟train是通过batch循环来一步步迭代自己的模型参数以优化模型性能；而在test阶段，模型已经定型，在输入不变时测试的结果应当保持恒定，是否还有必要循环测试呢？hhh

另外，不知道你是否了解过验证集和验证测试的作用，或许可以和现在的测试集对比一下，看看他们的区别和各自的用处优劣。

### 超参数的选择

实际上，梯度下降法的选取也算是一种超参数，即optimizer。不知你是否更改过超参数，尝试不同的效果，对比其性能？

### 结果

结果表现如何？为什么没有看到输出单元格hhh

### 其他学习目标

1. SExtractor处理的结果如何？
2. 不知你对复杂度、注意力机制、损失函数等的认识如何？后续我们再讨论一下吧